# Arguments

In [2]:

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
dataset_path = '../datasets/plantvillage/'
checkpoint_path = 'checkpoints-finetuning.hdf5'
input_width = 224
input_height = 224
input_depth = 3

batch_size = 32

# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))

#------------------------------------------
start_epoch = 0
num_of_epochs = 1
#------------------------------------------


Classes:  ['___Yellow_Leaf_Curl_Virus', '___Late_blight', '___Early_blight', '___Two-spotted_spider_mite', '___Septoria_leaf_spot', '___Bacterial_spot', '___healthy', '___Target_Spot', '___Mosaic_Virus', '___Leaf_Mold']
10


# Load model from disk

In [3]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)





# Print index of layers

In [4]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {}\t{}".format(i,layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D
[INFO] 19	Flatten
[INFO] 20	Dense
[INFO] 21	Dropout
[INFO] 22	Dense


# Unfreeze final CONV layers

In [5]:
for layer in model.layers[11:]:
    layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [6]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Load data

In [7]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
print("[INFO] loading images...")
image_paths = list(paths.list_images(dataset_path))

# Initial image preprocessing
aap = resizeImagePreprocessor(input_width, input_height)
iap= ImgToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)

[INFO] loading images...
[INFO]: Processed 500/18160
[INFO]: Processed 1000/18160
[INFO]: Processed 1500/18160
[INFO]: Processed 2000/18160
[INFO]: Processed 2500/18160
[INFO]: Processed 3000/18160
[INFO]: Processed 3500/18160
[INFO]: Processed 4000/18160
[INFO]: Processed 4500/18160
[INFO]: Processed 5000/18160
[INFO]: Processed 5500/18160
[INFO]: Processed 6000/18160
[INFO]: Processed 6500/18160
[INFO]: Processed 7000/18160
[INFO]: Processed 7500/18160
[INFO]: Processed 8000/18160
[INFO]: Processed 8500/18160
[INFO]: Processed 9000/18160
[INFO]: Processed 9500/18160
[INFO]: Processed 10000/18160
[INFO]: Processed 10500/18160
[INFO]: Processed 11000/18160
[INFO]: Processed 11500/18160
[INFO]: Processed 12000/18160
[INFO]: Processed 12500/18160
[INFO]: Processed 13000/18160
[INFO]: Processed 13500/18160
[INFO]: Processed 14000/18160
[INFO]: Processed 14500/18160
[INFO]: Processed 15000/18160
[INFO]: Processed 15500/18160
[INFO]: Processed 16000/18160
[INFO]: Processed 16500/18160
[INFO

# Checkpoints

In [8]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]

#load checkpoints if existing
import os


if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training Monitor

In [9]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Fit model

In [10]:
H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/1
425/425 [==============================] - 1993s 5s/step - loss: 0.4797 - accuracy: 0.8537 - val_loss: 0.1750 - val_accuracy: 0.9388

Epoch 00001: val_loss improved from inf to 0.17503, saving model to checkpoints-finetuning.hdf5


# Save model

In [11]:
model.save(model_save_path)

# Evaluate

In [12]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] evaluating after initialization...
                            precision    recall  f1-score   support

 ___Yellow_Leaf_Curl_Virus     0.9920    0.9288    0.9594       534
            ___Late_blight     0.9862    0.8106    0.8898       264
           ___Early_blight     0.9788    0.9726    0.9757       474
___Two-spotted_spider_mite     1.0000    0.8941    0.9441       236
     ___Septoria_leaf_spot     0.8462    0.6180    0.7143        89
         ___Bacterial_spot     0.8101    0.9676    0.8819       432
                ___healthy     0.8285    0.9128    0.8686       344
            ___Target_Spot     0.9143    0.8591    0.8858       447
           ___Mosaic_Virus     0.9628    0.9955    0.9789      1326
              ___Leaf_Mold     0.9974    0.9873    0.9923       394

               avg / total     0.9424    0.9388    0.9385      4540

